In [1]:
import h5py,numpy as np,os
path = '../data/fold0_noFIR.mat'
data = h5py.File(path, 'r')

x_train = data['trainX'][:].astype('float32')
x_train = np.expand_dims(x_train.transpose(),1)

x_val = data['valX'][:].astype('float32')
x_val = np.expand_dims(x_val.transpose(),1)

y_train = data['trainY'][:].astype('int32')
y_train = y_train.transpose()
y_train = y_train[:,0]
y_train[y_train<0] = 0

y_val = data['valY'][:].astype('int32')
y_val = y_val.transpose()
y_val = y_val[:,0]
y_val[y_val<0] = 0

val_parts = data['val_parts'][:].astype('int32').squeeze(0)
domain = np.array([x-97 for x in data['val_files'][:][0]])
train_domain = np.array([x-97 for x in data['train_files'][:][0]])

In [3]:
import torch.optim as optim
import torch
from torch.autograd import Variable
import time
import torch
import torch.nn as nn
# from torchvision import datasets, transforms
from torchsummary import summary
import numpy as np
from HeartCepTorch import MFCC_Gen,Network
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.nn.init as init
from torchsummary import summary

# from utils import log_macc, results_log
from dataLoader import reshape_folds
from sklearn.metrics import confusion_matrix
from keras.utils import to_categorical
import matplotlib.pyplot as plt
# import seaborn as sns
# import Evaluator
import dataLoader
# from custom_layers import Attention
from sklearn.model_selection import train_test_split
# import seaborn as sns
from collections import Counter
def to_numpy(x):
    return x.cpu().detach().numpy()
def plotf(x):
    plt.plot(to_numpy(x))

In [40]:
model = Network(2,0)
mfcc_gen = MFCC_Gen(fs=1000,filters=64,momentum=0.1)
model.cuda()
mfcc_gen.cuda()
class_criterion = nn.CrossEntropyLoss()
domain_criterion = nn.CrossEntropyLoss()
# model.load_state_dict(torch.load('../models/weights.0580-acc_0.7824-macc_0.8294.pt'))
model_path = '/media/mhealthra2/Data/heart_sound/Heartnet_Results/logs/gammatone_torch_layer/fold0_noFIR batch1500 continued_part4 .005_2020-04-17 03.18.55.957507/weights/'   
model.load_state_dict(torch.load(model_path+'/weights.0150-acc_0.7990-macc_0.8562.pt'))


<All keys matched successfully>

In [7]:
from torch.utils.data import Dataset, DataLoader

In [37]:
class HS_dataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.x[idx],self.y[idx]
dataset = HS_dataset(x_train,y_train)

In [19]:
# for validation set 
dataset = HS_dataset(x_val,y_val)

In [38]:
loader = DataLoader(dataset, batch_size=500, shuffle=False, sampler=None,
           batch_sampler=None, collate_fn=None,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None)

In [41]:
epoch_loss = 0
acc = 0
N = 0
y_true = None
feature = None
y_pred = None
model.eval()
mfcc_gen.eval()
with torch.no_grad():
    for (x,y) in iter(loader):
#         x,y = torch.from_numpy(x),torch.from_numpy(y)
        x = x.type(torch.FloatTensor).cuda()
        x = mfcc_gen(x)
        x = x.transpose(2,1)
        x = x.unsqueeze(1)
        x,y = Variable(x),Variable(y)
        y = y.long().cuda()
        cls= model(x).float()
        feat = model.extractor(x)
        
        if(feature is None):
            feature = to_numpy(feat)
        else:
            feature = np.concatenate((feature,to_numpy(feat)))
        if(y_pred is None):
            y_true = to_numpy(y)
        else:
            y_true = np.concatenate((y_true,to_numpy(y)))
        

In [43]:
feature.shape, feat.shape

((79810, 7168), torch.Size([310, 7168]))

In [42]:
train_feature = feature

In [22]:
val_feature = feature

In [65]:
from sklearn.ensemble import IsolationForest
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score

In [107]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 11)]

max_features = [0.7,0.8,1]

max_depth = [int(x) for x in np.linspace(10, 110, num = 2)]
max_depth.append(None)

min_samples_split = [2, 5]

min_samples_leaf = [1, 2]

bootstrap = [True, False]
max_samples = [0.1, 0.2]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               #'max_depth': max_depth,
               #'min_samples_split': min_samples_split,
               #'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [100, 290, 480, 670, 860, 1050, 1240, 1430, 1620, 1810, 2000], 'max_features': [0.7, 0.8, 1], 'bootstrap': [True, False]}


In [108]:
rf = IsolationForest()

In [115]:
help(IsolationForest)

Help on class IsolationForest in module sklearn.ensemble._iforest:

class IsolationForest(sklearn.base.OutlierMixin, sklearn.ensemble._bagging.BaseBagging)
 |  IsolationForest(n_estimators=100, max_samples='auto', contamination='auto', max_features=1.0, bootstrap=False, n_jobs=None, behaviour='deprecated', random_state=None, verbose=0, warm_start=False)
 |  
 |  Isolation Forest Algorithm.
 |  
 |  Return the anomaly score of each sample using the IsolationForest algorithm
 |  
 |  The IsolationForest 'isolates' observations by randomly selecting a feature
 |  and then randomly selecting a split value between the maximum and minimum
 |  values of the selected feature.
 |  
 |  Since recursive partitioning can be represented by a tree structure, the
 |  number of splittings required to isolate a sample is equivalent to the path
 |  length from the root node to the terminating node.
 |  
 |  This path length, averaged over a forest of such random trees, is a
 |  measure of normality and 

In [109]:
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 2,
                               cv = 2, verbose=1, random_state=42, n_jobs = -1
                              , scoring = {'Accuracy': make_scorer(accuracy_score)},
                              refit='Accuracy',return_train_score=True)

In [110]:
rf_random.fit(train_feature,y_train)

Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed: 204.6min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed: 204.6min finished


RandomizedSearchCV(cv=2, error_score=nan,
                   estimator=IsolationForest(behaviour='deprecated',
                                             bootstrap=False,
                                             contamination='auto',
                                             max_features=1.0,
                                             max_samples='auto',
                                             n_estimators=100, n_jobs=None,
                                             random_state=None, verbose=0,
                                             warm_start=False),
                   iid='deprecated', n_iter=2, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_features': [0.7, 0.8, 1],
                                        'n_estimators': [100, 290, 480, 670,
                                                         860, 1050, 1240, 1430,
                                                         1620, 

In [111]:
rff = rf_random.best_estimator_

In [135]:
rff = IsolationForest(n_estimators=1500,max_features=0.7,n_jobs=-1,verbose=1)

In [136]:
rff.fit(train_feature)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed: 37.7min remaining: 113.0min
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed: 37.8min finished


IsolationForest(behaviour='deprecated', bootstrap=False, contamination='auto',
                max_features=0.7, max_samples='auto', n_estimators=1500,
                n_jobs=-1, random_state=None, verbose=1, warm_start=False)

In [140]:
pred_val = rff.predict(val_feature)
print(pred_val.shape, y_val.shape)
acc = 100*np.sum(pred_val==y_val)/len(y_val)
max(acc,100-acc)

(6710,) (6710,)


56.944858420268254

In [141]:
pred_train = rff.predict(train_feature)
print(pred_train.shape, y_train.shape)
acc = 100*np.sum(pred_train==y_train)/len(y_train)
max(acc,100-acc)

(79810,) (79810,)


82.1087583009648

In [128]:
np.sum(pred_val[:]==y_val[:])

2890

In [132]:
print(pred_train.shape, y_train.shape)

(79810,) (79810,)


In [ ]:
# Number of trees in random forest
# Number of features to consider at every split
# Maximum number of levels in tree
# Minimum number of samples required to split a node
# Minimum number of samples required at each leaf node
# Method of selecting samples for training each tree
# Create the random grid
